In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

from lxml import etree
from analysis.xml import DocumentInfo, ElementInfo

In [2]:
tree = etree.parse('eda.als.xml')
root = tree.getroot()

In [3]:
doc = DocumentInfo(root)

In [8]:
elements_grouped_by_id = doc.get_elements_grouped_by_attribute("Id")
print(elements_grouped_by_id.keys())

dict_keys(['516', '10', '825', '16166', '822', '16108', '16052', '15965', '16124', '487', '16163', '16170', '15902', '16146', '16159', '15986', '8648', '15949', '16013', '16119', '15903', '15936', '16057', '16093', '837', '15858', '801', '4', '799', '15863', '15867', '8641', '841', '10585', '8649', '512', '16061', '16021', '15842', '15913', '15833', '15915', '16016', '847', '15883', '16001', '524', '15924', '16054', '843', '15995', '16063', '15952', '15897', '8632', '15956', '823', '16164', '16092', '16144', '15885', '8645', '16018', '17', '832', '15899', '517', '16173', '15865', '16134', '16176', '16131', '16077', '10582', '15935', '840', '10580', '510', '15969', '15854', '16014', '10595', '845', '15839', '16167', '505', '15917', '23', '15889', '12', '16', '830', '16085', '16110', '15997', '16028', '15911', '16002', '848', '15972', '16105', '15900', '16058', '16004', '8637', '16135', '523', '798', '16049', '16100', '15898', '16069', '818', '16095', '16038', '15966', '16155', '8635', '

In [9]:
print(elements_grouped_by_id["12"])

                 tag name content  parent index
7               MidiTrack    None             6
2652  RelativePathElement    None          2651
2680  RelativePathElement    None          2679
2702  RelativePathElement    None          2701
3024     ModulationTarget    None          3016
